In [1]:
import pandas as pd

In [2]:
date_range = pd.date_range(start=pd.datetime(year=2013,
                                             month=7,
                                             day=1),
                           end=pd.datetime(year=2019,
                                           month=4,
                                           day=27)
                          ).tolist()

def renamer(columns):
    new_names = {}
    for c in columns:
        new_names[c] = c.lower().replace('\n', '')
    return new_names

def find_total_column(columns):
    try:
        names = [s for s in columns if 'total passageiros' in s]
        assert len(names) == 1
    except AssertionError:
        names = [s for s in columns if 'tot passageiros' in s]
        assert len(names) == 1
            
    return names[0]

In [7]:
passenger_history = pd.DataFrame(columns=['8012', '8022'], index=date_range)
for date in date_range:
    try:
        spreadsheet = pd.read_excel('./raw/pass_transp_{year}_{month}_{day}.xls'.format(
            year = date.year,
            month = str(date.month).zfill(2),
            day = str(date.day).zfill(2)
        ), header=None)
    except AssertionError:
        continue

    # looking for the header row
    for i, row in spreadsheet.iterrows():
        if row.notnull().all():
            raw_data = spreadsheet.iloc[(i+1):].reset_index(drop=True)
            raw_data.columns = list(spreadsheet.iloc[i])
            break

    raw_data.dropna(inplace=True)
    raw_data.rename(columns=renamer(raw_data.columns), inplace=True)
    raw_data.rename(columns={find_total_column(raw_data.columns):'total'}, inplace=True)
    try:
        circular_1 = raw_data.loc[raw_data['linha'].str.contains('8012')]['total'].values[0]
        circular_2 = raw_data.loc[raw_data['linha'].str.contains('8022')]['total'].values[0]
    except IndexError:
        continue
    
    passenger_history.loc[date] = {
        '8012' : circular_1,
        '8022' : circular_2
    }

In [9]:
passenger_history

,8012,8022
2013-07-01,2969,2516
2013-07-02,3387,2869
2013-07-03,3361,3037
2013-07-04,3386,2996
2013-07-05,2909,2893
2013-07-06,783,1085
2013-07-07,308,366
2013-07-08,1376,1171
2013-07-09,431,473
2013-07-10,3397,2949


In [26]:
fails = max(passenger_history.isna().sum()['8012'], passenger_history.isna().sum()['8022']) / passenger_history.shape[0]
print('Failed on {:.2f}%.'.format(fails * 100))

Failed on 1.27%.


In [27]:
passenger_history.to_csv('./processed/pass_transp_clean.csv')